## Anomaly Sound Detection

We tried different models (Dense autoencoder, Variational autoencoder and Convolutional autoencoder). The last one gave us the best result, and the following notebook explains how we implemented this model. We ran the model locally, with some files that we were not able to upload on the kaggle platform. However, as we based our model architecture on the one provided by the repository https://github.com/AlexandrineRibeiro/DCASE-2020-Task-2, these files can be found there. 

In [ ]:
import os
import glob
import sys
import numpy
import itertools
import re
import csv
from tqdm import tqdm
from common_cae import *
import common_cae as com
import matplotlib.pyplot as plt
import numpy as np

import keras.models
from keras.models import Model
from keras.layers import Input, BatchNormalization, Activation, Reshape, Flatten
from keras.layers import Conv2D, Cropping2D, Conv2DTranspose, Dense
from keras.utils.vis_utils import plot_model
from keras.backend import int_shape
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import set_random_seed
from sklearn.externals.joblib import load, dump
from sklearn import metrics

## Define model: Convolutional autoencoder

In [ ]:
def get_data_shape(layer):
    return tuple(int_shape(layer)[1:])


def get_model(inputDim, latentDim):
    """
    define the keras model
    the model based on the simple convolutional auto encoder 
    """
    input_img = Input(shape=(inputDim[0], inputDim[1], 1))  # adapt this if using 'channels_first' image data format

    # encoder
    x = Conv2D(32, (5, 5),strides=(1,2), padding='same')(input_img)   #32x128 -> 32x64
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (5, 5),strides=(1,2), padding='same')(x)           #32x32
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (5, 5),strides=(2,2), padding='same')(x)          #16x16
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3),strides=(2,2), padding='same')(x)          #8x8
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3),strides=(2,2), padding='same')(x)          #4x4
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    volumeSize = int_shape(x)
    # at this point the representation size is latentDim i.e. latentDim-dimensional
    x = Conv2D(latentDim, (4,4), strides=(1,1), padding='valid')(x)
    encoded = Flatten()(x)
     
    # decoder
    x = Dense(volumeSize[1] * volumeSize[2] * volumeSize[3])(encoded) 
    x = Reshape((volumeSize[1], volumeSize[2], 512))(x)                #4x4

    x = Conv2DTranspose(256, (3, 3),strides=(2,2), padding='same')(x)  #8x8
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3),strides=(2,2), padding='same')(x)  #16x16   
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (5, 5),strides=(2,2), padding='same')(x)   #32x32
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(32, (5, 5),strides=(1,2), padding='same')(x)   #32x64
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    decoded = Conv2DTranspose(1, (5, 5),strides=(1,2), padding='same')(x) 

    return Model(inputs=input_img, outputs=decoded)


def load_model(file_path):
    return keras.models.load_model(file_path)


def plot(model):
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Visualize loss

In [ ]:
class visualizer(object):
    def __init__(self):
        import matplotlib.pyplot as plt
        self.plt = plt
        self.fig = self.plt.figure(figsize=(30, 10))
        self.plt.subplots_adjust(wspace=0.3, hspace=0.3)

    def loss_plot(self, loss, val_loss):
        """
        Plot loss curve.

        loss : list [ float ]
            training loss time series.
        val_loss : list [ float ]
            validation loss time series.

        return   : None
        """
        ax = self.fig.add_subplot(1, 1, 1)
        ax.cla()
        ax.plot(loss)
        ax.plot(val_loss)
        ax.set_title("Model loss")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend(["Train", "Validation"], loc="upper right")

    def save_figure(self, name):
        """
        Save figure.

        name : str
            save png file path.

        return : None
        """
        self.plt.savefig(name)

## Parameters

In [ ]:
param = com.yaml_load()
param

## Train

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,128), shuffle=True, step=8):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle

        self.data = np.load(self.list_IDs[0] , mmap_mode='r')
        
        self.step = step
        self.indexes_start = np.arange(self.data.shape[1]-self.dim[0]+self.step, step=self.step)
        self.max = len(self.indexes_start)
        self.indexes = np.arange(self.data.shape[0])
        
        self.indexes = np.repeat(self.indexes, self.max )
        self.indexes_start = np.repeat(self.indexes_start, self.data.shape[0])
    
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.data.shape[0] * self.max  / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        indexes_start = self.indexes_start[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes, indexes_start).reshape((self.batch_size, *self.dim, 1))

        return X, X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            np.random.shuffle(self.indexes_start)


    def __data_generation(self, indexes, index_start):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))

        # Generate data
        for i, (id_file, id_start) in enumerate(zip(indexes, index_start)):

            x = self.data[id_file,]
            length, mels = x.shape

            start = id_start

            start = min(start, length - self.dim[0])
            
            # crop part of sample
            crop = x[start:start+self.dim[0], :]

            X[i,] = crop
        return X

In [ ]:
# check mode
# "development": mode == True
# "evaluation": mode == False
mode, target = True, "slider"

# fix a random seed
set_random_seed(88)

if mode is None:
    sys.exit(-1)

# make output directory
os.makedirs(param["model_directory"], exist_ok=True)

# initialize the visualizer
visualizer = visualizer()

# load base_directory list
dirs = com.select_dirs(param=param, mode=mode, target=target)


# loop of the base directory (machine types)
for idx, target_dir in enumerate(dirs):
    print("\n===========================")
    print("[{idx}/{total}] {dirname}".format(dirname=target_dir, idx=idx+1, total=len(dirs)))

    # set path
    machine_type = os.path.split(target_dir)[1]
    model_file_path = "{model}/model_{machine_type}.hdf5".format(model=param["model_directory"],
                                                                machine_type=machine_type)
    best_model_filepath = "{model}/bestmodel_{machine_type}_".format(model=param["model_directory"],
                                                                machine_type=machine_type)
    history_img = "{model}/history__{machine_type}.png".format(model=param["model_directory"],
                                                                machine_type=machine_type)
    features_file_path = "{features}/{machine_type}".format(features=param["features_directory"],
                                                                machine_type=machine_type)
    features_dir_path = os.path.abspath(features_file_path)


    if os.path.exists(model_file_path):
        com.logger.info("model exists")
        continue


    # get features
    # get npy files list (features files)
    list_files_npy_train = file_list_generator(features_dir_path, dir_name="train", ext="npy")
    list_files_npy_val = file_list_generator(features_dir_path, dir_name="val", ext="npy")

    if len(list_files_npy_train)==0 or len(list_files_npy_val)==0:
        com.logger.exception("no_npy_files!!")
        sys.exit(-1)  


    shape0_feat = param["autoencoder"]["shape0"]
    shape1_feat = param["feature"]["n_mels"]

    # load data 
    gen_train = DataGenerator(list_files_npy_train, batch_size=param["fit"]["batch_size"], dim=(shape0_feat,shape1_feat), step=param["step"])
    gen_val = DataGenerator(list_files_npy_val,  batch_size=param["fit"]["batch_size"], dim=(shape0_feat,shape1_feat), shuffle=False, step=param["step"])


    # train model
    print("============== MODEL TRAINING ==============")

    # checkpoint
    model_checkpoint = ModelCheckpoint(best_model_filepath+"{epoch:02d}.hdf5", monitor='val_loss', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor='val_loss', mode='min', patience=10, min_delta=0.0001)

    # create model
    model = get_model((shape0_feat, shape1_feat), param["autoencoder"]["latentDim"])
    model.summary()


    #train model
    model.compile(**param["fit"]["compile"])
    history = model.fit_generator(gen_train, 
                        validation_data=gen_val,
                        epochs=param["fit"]["epochs"], 
                        verbose=param["fit"]["verbose"],
                        callbacks=[model_checkpoint, early])

    visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
    visualizer.save_figure(history_img)
    model.save(model_file_path)
    com.logger.info("save_model -> {}".format(model_file_path))
    print("============== END TRAINING ==============")

## Test

In [ ]:
# useful functions
def save_csv(save_file_path,
             save_data):
    with open(save_file_path, "w", newline="") as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(save_data)


def get_machine_id_list_for_test(target_dir,
                                 dir_name="test",
                                 ext="wav"):
    # create test files
    dir_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    file_paths = sorted(glob.glob(dir_path))
    # extract id
    machine_id_list = sorted(list(set(itertools.chain.from_iterable(
        [re.findall('id_[0-9][0-9]', ext_id) for ext_id in file_paths]))))
    return machine_id_list


def test_file_list_generator(target_dir,
                             id_name,
                             dir_name="test",
                             prefix_normal="normal",
                             prefix_anomaly="anomaly",
                             ext="wav"):
    com.logger.info("target_dir : {}".format(target_dir+"_"+id_name))

    # development
    if mode:
        normal_files = sorted(
            glob.glob("{dir}/{dir_name}/{prefix_normal}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                 dir_name=dir_name,
                                                                                 prefix_normal=prefix_normal,
                                                                                 id_name=id_name,
                                                                                 ext=ext)))
        normal_labels = numpy.zeros(len(normal_files))
        anomaly_files = sorted(
            glob.glob("{dir}/{dir_name}/{prefix_anomaly}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                  dir_name=dir_name,
                                                                                  prefix_anomaly=prefix_anomaly,
                                                                                  id_name=id_name,
                                                                                  ext=ext)))
        anomaly_labels = numpy.ones(len(anomaly_files))
        files = numpy.concatenate((normal_files, anomaly_files), axis=0)
        labels = numpy.concatenate((normal_labels, anomaly_labels), axis=0)
        com.logger.info("test_file  num : {num}".format(num=len(files)))
        if len(files) == 0:
            com.logger.exception("no_wav_file!!")
        print("\n========================================")

    # evaluation
    else:
        files = sorted(
            glob.glob("{dir}/{dir_name}/*{id_name}*.{ext}".format(dir=target_dir,
                                                                  dir_name=dir_name,
                                                                  id_name=id_name,
                                                                  ext=ext)))
        labels = None
        com.logger.info("test_file  num : {num}".format(num=len(files)))
        if len(files) == 0:
            com.logger.exception("no_wav_file!!")
        print("\n=========================================")

    return files, labels

In [ ]:
# check mode
# "development": mode == True
# "evaluation": mode == False
mode, target = True, "slider"
if mode is None:
    sys.exit(-1)

# make output result directory
os.makedirs(param["result_directory"], exist_ok=True)

# load base directory
dirs = com.select_dirs(param=param, mode=mode, target=target)

# initialize lines in csv for AUC and pAUC
csv_lines = []

# loop of the base directory (machine type)
for idx, target_dir in enumerate(dirs):
    print("\n===========================")
    print("[{idx}/{total}] {dirname}".format(dirname=target_dir, idx=idx+1, total=len(dirs)))
    machine_type = os.path.split(target_dir)[1]

    print("============== MODEL LOAD ==============")
    # set model path
#     model_file = "{model}/model_{machine_type}.hdf5".format(model=param["model_directory"],
#                                                             machine_type=machine_type)
    model_file = "./model/bestmodel_slider_13.hdf5"

    features_file_path = "{features}/{machine_type}/{tip}".format(features=param["features_directory"],
                                                                    machine_type=machine_type, tip="test")
    features_dir_path = os.path.abspath(features_file_path)

    #load scaler
    scaler_file_path = "{scalers}/{machine_type}".format(scalers=param["scalers_directory"], machine_type=machine_type)
    scaler_file_path = os.path.abspath(scaler_file_path)
    scaler = load(scaler_file_path+"/scaler_{machine_type}.bin".format(machine_type=machine_type))


    # load model file
    if not os.path.exists(model_file):
        com.logger.error("{} model not found ".format(machine_type))
        #sys.exit(-1)
        continue
    model = load_model(model_file)
    model.summary()


    if mode:
        # results by type
        csv_lines.append([machine_type])
        csv_lines.append(["id", "AUC", "pAUC"])
        performance = []

    machine_id_list = get_machine_id_list_for_test(target_dir)

    # loop of the machine type directory (machine id)
    for id_str in machine_id_list:
        # load test file
        test_files, y_true = test_file_list_generator(target_dir, id_str)

        # setup anomaly score file path
        anomaly_score_csv = "{result}/anomaly_score_{machine_type}_{id_str}.csv".format(
                                                                                 result=param["result_directory"],
                                                                                 machine_type=machine_type,
                                                                                 id_str=id_str)
        anomaly_score_list = []

        print("\n============== BEGIN TEST FOR A MACHINE ID ==============")
        y_pred = [0. for k in test_files]
        for file_idx, file_path in tqdm(enumerate(test_files), total=len(test_files)):

            try:
                # get audio features
                vector_array = com.file_to_vector_array(file_path, None, scaler,
                                                n_mels=param["feature"]["n_mels"],
                                                frames=param["feature"]["frames"],
                                                n_fft=param["feature"]["n_fft"],
                                                hop_length=param["feature"]["hop_length"],
                                                power=param["feature"]["power"])

                length, _ = vector_array.shape

                dim = param["autoencoder"]["shape0"]
                step = param["step"]

                idex = numpy.arange(length-dim+step, step=step)

                for idx in range(len(idex)):
                    start = min(idex[idx], length - dim)

                    vector = vector_array[start:start+dim,:]

                    vector = vector.reshape((1, vector.shape[0], vector.shape[1]))
                    if idx==0:
                        batch = vector
                    else:
                        batch = numpy.concatenate((batch, vector))


                # add channels dimension
                data = batch.reshape((batch.shape[0], batch.shape[1], batch.shape[2], 1))

                # calculate predictions
                errors = numpy.mean(numpy.square(data - model.predict(data)), axis=-1)

                y_pred[file_idx] = numpy.mean(errors)
                anomaly_score_list.append([os.path.basename(file_path), y_pred[file_idx]])


            except:
                com.logger.error("file broken!!: {}".format(file_path))
                sys.exit(-1)

        # save anomaly score
        save_csv(save_file_path=anomaly_score_csv, save_data=anomaly_score_list)
        com.logger.info("anomaly score result ->  {}".format(anomaly_score_csv))

        if mode:
            # append AUC and pAUC to lists
            auc = metrics.roc_auc_score(y_true, y_pred)
            p_auc = metrics.roc_auc_score(y_true, y_pred, max_fpr=param["max_fpr"])
            csv_lines.append([id_str.split("_", 1)[1], auc, p_auc])
            performance.append([auc, p_auc])
            com.logger.info("AUC : {}".format(auc))
            com.logger.info("pAUC : {}".format(p_auc))

        print("\n============ END OF TEST FOR A MACHINE ID ============")

    if mode:
        # calculate averages for AUCs and pAUCs
        averaged_performance = numpy.mean(numpy.array(performance, dtype=float), axis=0)
        com.logger.info("average AUC : {}".format(averaged_performance[0]))
        com.logger.info("average pAUC : {}".format(averaged_performance[1]))
        csv_lines.append(["Average"] + list(averaged_performance))
        csv_lines.append([])

if mode:
    # output results
    if target:
        result_path = "{result}/{target}_{file_name}".format(result=param["result_directory"], file_name=param["result_file"], target=target)
    else:
        result_path = "{result}/{file_name}".format(result=param["result_directory"], file_name=param["result_file"])
    com.logger.info("AUC and pAUC results -> {}".format(result_path))
    save_csv(save_file_path=result_path, save_data=csv_lines)
    print("Test process for dev set finished!")

if not mode:
    df1 = pd.read_csv('./result/anomaly_score_slider_id_01.csv', delimiter=',', header=None)
    df3 = pd.read_csv('./result/anomaly_score_slider_id_03.csv', delimiter=',', header=None)
    df5 = pd.read_csv('./result/anomaly_score_slider_id_05.csv', delimiter=',', header=None)

    dfs = [df1, df3, df5]
    eval_df = pd.concat(dfs)
    eval_df = eval_df.rename(columns={0: "file_name", 1: "anomaly_score"})

    eval_df.to_csv('./result/baseline_submission.csv', index=False)
    print("Test process for eval set finished!")